## Inference
We have tested a variety of procedures:
* Inference in each frame (slow)
* Batch processing every 5 or 20 (slow)
* Inference every 20 frames (fastest)

In [2]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import tensorflow as tf
from tensorflow import keras
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
# We want the pose estimation
mp_pose = mp.solutions.pose

keras.backend.set_learning_phase(0)

2022-01-15 00:34:34.662405: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-01-15 00:34:34.662473: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/home/mauriciomani/Desktop/rowing_tracker/venv/lib/python3.8/site-packages/tensorflow/python/keras/backend.py:434: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn('`tf.keras.backend.set_learning_phase` is deprecated and '


In [3]:
model = load_model("../models/mobile.h5")

2022-01-15 00:34:37.416445: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-01-15 00:34:37.423164: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-01-15 00:34:37.465172: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-15 00:34:37.465868: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce GTX 1650 Ti with Max-Q Design computeCapability: 7.5
coreClock: 1.2GHz coreCount: 16 deviceMemorySize: 3.82GiB deviceMemoryBandwidth: 149.04GiB/s
2022-01-15 00:34:37.466087: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file

In [7]:
### Video
cap = cv2.VideoCapture('../videos/rowing2.mp4')

count = 0
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:  
    while cap.isOpened():
        ret, frame = cap.read()
        
        cv_image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        cv_image.flags.writeable = False
        
        results = pose.process(cv_image)
        
        cv_image.flags.writeable = True
        cv_image = cv2.cvtColor(cv_image, cv2.COLOR_RGB2BGR)
        
        try:
            landmarks = results.pose_landmarks.landmark
            right_shoulder = landmarks[12]
            right_hip = landmarks[24]
            left_shoulder = landmarks[11]
            left_hip = landmarks[23]
            nose = landmarks[0]
            max_x = max(right_shoulder.x, left_shoulder.x, right_hip.x, left_hip.x)
            max_x = max_x + (max_x * 0.2)
            min_x = min(right_shoulder.x, left_shoulder.x, right_hip.x, left_hip.x)
            min_x = min_x - (min_x * 0.2)
            min_y = min(right_hip.y, left_hip.y)
            min_y = min_y + (min_y * 0.1)
            max_y = nose.y - (nose.y * 0.2)
            cropped_img = cv_image[int(max_y*720):int(min_y*720), int(min_x*1280):int(max_x*1280)]
            image_cnn = cv2.resize(cropped_img, (224, 224))
            img_array = image.img_to_array(image_cnn)
            img_batch = np.expand_dims(img_array, axis=0)
            processed_image = tf.keras.applications.mobilenet.preprocess_input(img_batch)
        
            #prediction = model.predict(x=processed_image)
            if (count % 10 == 0):
                prediction = model.predict(x=processed_image)[0][1]
                if prediction >= 0.5:
                    print("Warning")
        except:
            print("Image problems")
            pass
        
        count += 1
        
        cv2.imshow("mediapipe test", cv_image)#image_cnn
        if cv2.waitKey(10) & 0xFF == ord("q"):
            break
        
    
    cap.release()
    cv2.destroyAllWindows()

Warning
Warning
Warning
Warning
Image problems
Image problems
Image problems
Image problems
Image problems
Warning
Warning
Warning
Warning
Warning
Warning
Warning
Warning
Warning
Warning
Warning
Warning
Warning
Warning
Warning
Image problems
Warning
Warning
Warning
Warning
Warning
Warning
Warning
Warning
Warning
Warning
Warning
Warning
Warning
Warning
Warning
Warning
Warning
Image problems
Image problems
Image problems
Image problems
Warning
Warning
Warning
